In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skimage.feature import canny
import pandas as pd
import pywt

# Function to apply a steerable filter (example using a simple Gaussian kernel for demonstration)
def steerable_filter(image):
    # Using a simple Gaussian kernel for demonstration purposes
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getGaussianKernel(3, 0.5)
    filtered_image = cv2.filter2D(gray, -1, kernel)
    return filtered_image.flatten()

def extract_hsv(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    hist = []
    for channel in [h, s, v]:
        hist_channel = cv2.calcHist([channel], [0], None, [8], [0, 256])
        hist_channel = hist_channel.flatten() / np.sum(hist_channel)
        hist.append(hist_channel)
    return np.concatenate(hist)

def extract_edge(image):
    # Using steerable filter instead of Canny for edge extraction
    edges = steerable_filter(image)
    return edges

def extract_pyramid(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coeffs = pywt.dwt2(gray, 'bior1.3')
    LL, (LH, HL, HH) = coeffs
    return np.concatenate([LL.flatten(), LH.flatten(), HL.flatten(), HH.flatten()])

def load_features(dataset_path, feature_set, image_size=(128, 128)):
    features = []
    labels = []
    class_folders = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
    for label, class_name in enumerate(tqdm(class_folders, desc="Classes")):
        class_path = os.path.join(dataset_path, class_name)
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for img_file in tqdm(image_files, desc=f"Extracting {class_name}", leave=False):
            img_path = os.path.join(class_path, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, image_size)
            feat = []
            if 'HSV' in feature_set:
                feat.append(extract_hsv(image))
            if 'Edge' in feature_set:
                feat.append(extract_edge(image))
            if 'Pyramid' in feature_set:
                feat.append(extract_pyramid(image))
            combined = np.concatenate(feat)
            features.append(combined)
            labels.append(label)
    return np.array(features), np.array(labels)

dataset_path = r"D:\git\FYP\Python\TwoStage\data\Dataset1\Pandora7k_original"

feature_sets = [
    'Edge',
    'HSV',
    'Pyramid',
    'Pyramid+Edge',
    'Pyramid+Edge+HSV'
]

results = {}

print("\nStarting feature extraction and classification...\n")
for fset in feature_sets:
    print(f"\nProcessing Feature Set: {fset}")
    X, y = load_features(dataset_path, fset.split('+'))

    print("Splitting dataset...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    classifiers = {
        'KNN (k=1)': KNeighborsClassifier(n_neighbors=1),
        'KNN (k=10)': KNeighborsClassifier(n_neighbors=10),
        'Naive Bayes': GaussianNB()
    }

    results[fset] = {}
    for clf_name, clf in classifiers.items():
        print(f"Training {clf_name}...")
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred) * 100
        results[fset][clf_name] = acc
        print(f"{clf_name} Accuracy: {acc:.2f}%")

print("\nFinal Results Table (Accuracy %):")
print(f"{'Feature Set':<25}{'KNN (k=1)':<12}{'KNN (k=10)':<12}{'Naive Bayes':<12}")
for fset in feature_sets:
    row = f"{fset:<25}"
    for clf in ['KNN (k=1)', 'KNN (k=10)', 'Naive Bayes']:
        row += f"{results[fset][clf]:<12.2f}"
    print(row)

print("\nSaving results to Excel...")
df = pd.DataFrame.from_dict(results, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Feature Set'}, inplace=True)
output_path = 'results_table.xlsx'
df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")



Starting feature extraction and classification...


Processing Feature Set: Edge


Classes: 100%|██████████| 12/12 [02:12<00:00, 11.04s/it]


Splitting dataset...
Training KNN (k=1)...
KNN (k=1) Accuracy: 18.19%
Training KNN (k=10)...
KNN (k=10) Accuracy: 20.01%
Training Naive Bayes...
Naive Bayes Accuracy: 21.77%

Processing Feature Set: HSV


Classes: 100%|██████████| 12/12 [00:49<00:00,  4.16s/it]


Splitting dataset...
Training KNN (k=1)...
KNN (k=1) Accuracy: 29.69%
Training KNN (k=10)...
KNN (k=10) Accuracy: 34.37%
Training Naive Bayes...
Naive Bayes Accuracy: 32.36%

Processing Feature Set: Pyramid


...

Splitting dataset...
Training KNN (k=1)...
KNN (k=1) Accuracy: 18.26%
Training KNN (k=10)...
KNN (k=10) Accuracy: 19.49%
Training Naive Bayes...
Naive Bayes Accuracy: 32.49%

Processing Feature Set: Pyramid+Edge


Classes: 100%|██████████| 12/12 [00:51<00:00,  4.31s/it]


Splitting dataset...
Training KNN (k=1)...
KNN (k=1) Accuracy: 18.13%
Training KNN (k=10)...
KNN (k=10) Accuracy: 19.49%
Training Naive Bayes...
Naive Bayes Accuracy: 28.65%

Processing Feature Set: Pyramid+Edge+HSV


Classes: 100%|██████████| 12/12 [00:52<00:00,  4.35s/it]


Splitting dataset...
Training KNN (k=1)...
KNN (k=1) Accuracy: 18.13%
Training KNN (k=10)...
KNN (k=10) Accuracy: 19.49%
Training Naive Bayes...
Naive Bayes Accuracy: 29.37%

Final Results Table (Accuracy %):
Feature Set              KNN (k=1)   KNN (k=10)  Naive Bayes 
Edge                     18.19       20.01       21.77       
HSV                      29.69       34.37       32.36       
Pyramid                  18.26       19.49       32.49       
Pyramid+Edge             18.13       19.49       28.65       
Pyramid+Edge+HSV         18.13       19.49       29.37       

Saving results to Excel...
Results saved to results_table.xlsx
